# A YOLOv3 Object Detector with Darknet in the Cloud (GPU ENABLED)
This notebook copies over training and validation data, configuration files, and code needed to train a YOLOv3 network.   It is set up to classify and locate a single class ("Box").

# Step 1: Cloning the repository
The following cells will clone a repository from Erik Lindernoren.

In [1]:
# clone darknet repo
#!git clone https://github.com/AlexeyAB/darknet
!git clone https://github.com/eriklindernoren/PyTorch-YOLOv3

Cloning into 'PyTorch-YOLOv3'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1032 (delta 36), reused 49 (delta 28), pack-reused 965
Receiving objects: 100% (1032/1032), 16.25 MiB | 4.49 MiB/s, done.
Resolving deltas: 100% (596/596), done.


# The next cell just makes sure that all the requirements for the code are installed.

In [2]:
%cd /content/PyTorch-YOLOv3
!pwd
!sudo pip3 install -r requirements.txt
#!sudo pip install imgaug

import torch
#import imgaug
#from imgaug import *
#from imgaug.augmentables.segmaps import SegmentationMapsOnImage


/content/PyTorch-YOLOv3
/content/PyTorch-YOLOv3
     |████████████████████████████████| 952kB 13.7MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=49e774ef5e60f74e49a8a4876a7b5088d17c9df53c8223a779e50d99c3723100
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


# Step 2:  Download weights from Coco training if desired

In [3]:
# If needed, you can Download some pre-trained weights from Coco.  Not necessary in this case
#
%cd ..
%cd PyTorch-YOLOv3
%cd weights

!bash download_weights.sh
%cd ..

/content
/content/PyTorch-YOLOv3
/content/PyTorch-YOLOv3/weights
--2021-03-20 03:19:38--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  3.64MB/s    in 29s     

2021-03-20 03:20:07 (8.29 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2021-03-20 03:20:07--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  8.50MB/s    in 5.3s    

2021-03-20 03:20:13 (6.

In [4]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


# Step 3:  Mount your Google Drive, and upload data into the Cloud

In [5]:
!ls
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ls

assets	data	   LICENSE    README.md		test.py   utils
config	detect.py  models.py  requirements.txt	train.py  weights
/content
Mounted at /content/gdrive
gdrive	PyTorch-YOLOv3	sample_data


In [6]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'   CS231A_project  'My Drive'	 obj.data   yolov3   yolov4


In [8]:
%cd /content/PyTorch-YOLOv3/data/custom/
!rm images/train.jpg  # Left over from github install (not wanted)
#!ls images
#!cp /mydrive/CS231A_project/Data/train_3D_yolov3/custom_images.zip .
#!cp /mydrive/CS231A_project/Data/train_3D_yolov3/custom_labels.zip .
!cp /mydrive/CS231A_project/Data/train_3D_yolov3/custom_images_hot.zip .
!cp /mydrive/CS231A_project/Data/train_3D_yolov3/custom_labels_hot.zip .
!unzip -nq custom_images_hot.zip
!unzip -nq custom_labels_hot.zip

%mkdir /content/PyTorch-YOLOv3/data/custom/validation
%cd /content/PyTorch-YOLOv3/data/custom/validation/
!cp /mydrive/CS231A_project/Data/test_3D_yolov3/custom_images.zip .
!cp /mydrive/CS231A_project/Data/test_3D_yolov3/custom_labels.zip .
!unzip -nq custom_images.zip
!unzip -nq custom_labels.zip

%cd /content/PyTorch-YOLOv3/
!cp /mydrive/yolov3/generate*.py .
# Above copies the generate_train.py and generate_test.py to the cloud

!python generate_train.py
!python generate_test.py
# Generate the train.txt and valid.txt files in data/custom


/content/PyTorch-YOLOv3/data/custom
rm: cannot remove 'images/train.jpg': No such file or directory
mkdir: cannot create directory ‘/content/PyTorch-YOLOv3/data/custom/validation’: File exists
/content/PyTorch-YOLOv3/data/custom/validation
/content/PyTorch-YOLOv3


# Make sure that the label annotations do not have any comma's.

In [9]:
%cd /content/PyTorch-YOLOv3/
!pwd

!cp /mydrive/yolov3/comm_out_labels.py .
!cp /mydrive/yolov3/comm_out_valid_labels.py .
#!ls data/custom/labels/train_image_00067.txt*
!python comm_out_labels.py
!python comm_out_valid_labels.py

/content/PyTorch-YOLOv3
/content/PyTorch-YOLOv3


In [7]:
%cd /content/PyTorch-YOLOv3/config/
#!bash create_custom_model.sh <num-classes> # Will create custom model 'yolov3-custom.cfg'
!bash create_custom_model.sh 1 # Will create custom model 'yolov3-custom.cfg' for a single class

# But, this will be overwritten by a saved copy in next block
%cd ../


/content/PyTorch-YOLOv3/config
/content/PyTorch-YOLOv3


In [10]:
# This assumes that a custom yolov3-custom.cfg file has been generated 
#     and edited and left in /mydrive/yolov3
%cd /content/PyTorch-YOLOv3/
!cp /mydrive/yolov3/classes.names data/custom/.

%cd /content/PyTorch-YOLOv3/config
!cp /mydrive/yolov3/yolov3-custom.cfg .
#!ls -al data/custom/classes.names
#!more data/custom/classes.names
!ls -al
#!cp yolov3-custom.cfg /mydrive/yolov3/.
%cd /content/PyTorch-YOLOv3/

/content/PyTorch-YOLOv3
/content/PyTorch-YOLOv3/config
total 56
drwxr-xr-x 2 root root 4096 Mar 20 03:23 .
drwxr-xr-x 8 root root 4096 Mar 20 03:23 ..
-rw-r--r-- 1 root root  115 Mar 20 03:19 coco.data
-rw-r--r-- 1 root root 8532 Mar 20 03:19 create_custom_model.sh
-rw-r--r-- 1 root root   99 Mar 20 03:19 custom.data
-rw-r--r-- 1 root root 8338 Mar 20 03:19 yolov3.cfg
-rw------- 1 root root 8368 Mar 20 03:23 yolov3-custom.cfg
-rw-r--r-- 1 root root 2025 Mar 20 03:19 yolov3-tiny.cfg
/content/PyTorch-YOLOv3


# Step 4:  Get ready for training.  First upload modified code.

In [11]:
!cp /mydrive/yolov3/train.py train.py
!cp /mydrive/yolov3/detect.py detect.py
!ls -al train.py

-rw-r--r-- 1 root root 7585 Mar 20 03:23 train.py


In [20]:
!tensorboard --logdir='logs' --port=6006

2021-03-19 14:06:56.064520: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Step 5:  Then Training can begin!

In [13]:
#  To start training on YOLOv3
#
%cd /content/PyTorch-YOLOv3

!python3 train.py --model_def config/yolov3-custom.cfg --data_config config/custom.data

!cp weights/wgts_50_74 /mydrive/yolov3/wgts_50_hot_74
!cp weights/wgts_50_all /mydrive/yolov3/wgts_50_hot_all

/content/PyTorch-YOLOv3
Namespace(batch_size=8, checkpoint_interval=1, compute_map=False, data_config='config/custom.data', epochs=50, evaluation_interval=1, gradient_accumulations=2, img_size=416, logdir='logs', model_def='config/yolov3-custom.cfg', multiscale_training=True, n_cpu=8, pretrained_weights=None, verbose=False)
2021-03-20 03:47:19.342881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Training Epoch 0: 100% 101/101 [01:55<00:00,  1.14s/it]

---- Evaluating

# The previous block will save weights to the Google Drive so that all is not lost when Google disconnects this session from the Cloud.

#
#
# *The next section is for running detection on sample images.   It uses the last checkpoint (weights) of the training.*

In [34]:
!python3 detect.py --model_def config/yolov3-custom.cfg --class_path data/custom/classes.names  --conf_thres=0.4 --checkpoint_model /mydrive/yolov3/checkpoints/yolov3_ckpt_49.pth --image_folder /mydrive/yolov3/test/samples --weights_path /mydrive/yolov3/checkpoints/yolov3_ckpt_49.pth
#!python3 detect.py --model_def config/yolov3-custom.cfg --class_path data/classes.names --weights_path /mydrive/yolov3/wgts_50_74.weights  --checkpoint_model /mydrive/yolov3/checkpoints/yolov3_ckpt_49.pth --image_folder /mydrive/yolov3/test/samples

Namespace(batch_size=1, checkpoint_model='/mydrive/yolov3/checkpoints/yolov3_ckpt_49.pth', class_path='data/custom/classes.names', conf_thres=0.4, image_folder='/mydrive/yolov3/test/samples', img_size=416, model_def='config/yolov3-custom.cfg', n_cpu=0, nms_thres=0.4, weights_path='/mydrive/yolov3/checkpoints/yolov3_ckpt_49.pth')

Performing object detection:
	+ Batch 0, Inference Time: 0:00:00.096581
	+ Batch 1, Inference Time: 0:00:00.083676
	+ Batch 2, Inference Time: 0:00:00.082740
	+ Batch 3, Inference Time: 0:00:00.078909

Saving images:
(0) Image: '/mydrive/yolov3/test/samples/SEQ_image_4.jpg'
(1) Image: '/mydrive/yolov3/test/samples/test_image_00004.jpg'
	+ Label: Box, Conf: 0.99999
(2) Image: '/mydrive/yolov3/test/samples/test_image_00005.jpg'
	+ Label: Box, Conf: 1.00000
(3) Image: '/mydrive/yolov3/test/samples/test_image_00006.jpg'
	+ Label: Box, Conf: 0.99999


In [54]:
#  This is junk
#
#%cd /mydrive/yolov3/
#!ls -al *.py

#!python Add1000filename.py

#!cp /mydrive/CS231A_project/Data/train_3D_yolov3_hot/train*.jpg /mydrive/CS231A_project/Data/train_3D_yolov3/images/.
#!cp /mydrive/CS231A_project/Data/train_3D_yolov3_hot/train*.txt /mydrive/CS231A_project/Data/train_3D_yolov3/labels/.

#%cd /content/PyTorch-YOLOv3/data/custom/
#!cp /mydrive/CS231A_project/Data/train_3D_yolov3/custom_images.zip .
%cd /mydrive/CS231A_project/Data/train_3D_yolov3/
#%cd /mydrive/CS231A_project/Data/train_3D_yolov3/Data/custom/
#!pwd
#!zip -rq custom_images_hot.zip images
#!zip -rq custom_labels_hot.zip labels

/content/gdrive/My Drive/CS231A_project/Data/train_3D_yolov3
/content/gdrive/My Drive/CS231A_project/Data/train_3D_yolov3
